## 📦 Packages and Basic Setup
---

In [ ]:
%%capture
import os

import torch

torch_version = torch.__version__.split("+")
os.environ["TORCH"] = torch_version[0]
os.environ["CUDA"] = torch_version[1]

!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{CUDA}.html
!pip install torch-geometric
!pip install -q --upgrade wandb

import torch.nn.functional as F
import torch_geometric.transforms as T
import wandb
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import ChebConv

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# @title ⚙ Configuration
from google.colab import userdata

# Paste your api key here
os.environ["WANDB_API_KEY"] = userdata.get("W&B")

wandb.init(project="ChebGCN", entity="graph-neural-networks")

config = wandb.config
config.lr = 0.01  # @param {type: "number"}
config.num_hops = 3  # @param {type: "number"}
config.latent_dim = 64  # @param {type: "number"}
config.num_epochs = 50  # @param {type: "number"}
wandb.config.update(config)

## 💿 The Dataset
---

In [ ]:
dataset = Planetoid("data/", "Cora", transform=T.NormalizeFeatures())
data = dataset[0]

## ✍️ Model Architecture & Training
---

In [ ]:
class ChebNet(torch.nn.Module):
    def __init__(self, num_features, latent_dim, num_classes, num_hops):
        super().__init__()
        self.conv1 = ChebConv(num_features, latent_dim, num_hops)
        self.conv2 = ChebConv(latent_dim, num_classes, num_hops)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


model = ChebNet(
    num_features=dataset.num_features,
    latent_dim=config.latent_dim,
    num_classes=dataset.num_classes,
    num_hops=config.num_hops,
)

model, data = model.to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=5e-4)

## Training
---

In [ ]:
def train(model, optimizer, data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def evaluate(model, data):
    model.eval()
    out = model(data)

    outs = {}
    for key in ["train", "val", "test"]:
        mask = data[f"{key}_mask"]
        loss = float(F.nll_loss(out[mask], data.y[mask]))
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()

        outs[f"{key}_loss"] = loss
        outs[f"{key}_acc"] = acc

    return outs

In [ ]:
for epoch in range(1, config.num_epochs + 1):
    train(model, optimizer, data)
    eval_info = evaluate(model, data)
    eval_info["epoch"] = epoch

    wandb.log(eval_info)

wandb.finish()